In [1]:
!pip install -U datasets -q

In [2]:
from datasets import load_dataset

In [3]:
imdb_dataset = load_dataset('imdb')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
device = "cuda" if torch.cuda.is_available else "cpu"

model_name = 'EKKam/opt-1.5b_imdb_sft'
# model_name = 'Qwen/Qwen2.5-1.5B-Instruct'
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
import torch

device = "cuda" if torch.cuda.is_available else "cpu"
# Sample input batch
batch_texts = [
    "Once upon a time there was a ghost in the town",
    "In a galaxy far, far away there lies a star that is ",
    "He was a good boy. He did all his homework in time "
]

# Tokenize input batch and prepare for generation
inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, padding_side='left', max_length=5).to(device)

# Generate sequences in batch
# You can customize parameters such as max_length, num_return_sequences, etc.
# generated_outputs = model.generate(
#     **inputs,
#     max_length=512,

# )

In [7]:
decoded_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in generated_outputs]

In [8]:
pad_token = torch.tensor(tokenizer.encode(tokenizer.pad_token)).to(device)

In [10]:
pad_token

tensor([2, 0], device='cuda:0')

In [11]:
tokenizer.pad_token_id

0

In [14]:
attention_mask = generated_outputs.ne(torch.tensor([tokenizer.pad_token_id]).to(device))

In [ ]:
tokenizer.decode(generated_outputs[0])

In [44]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [15]:
attention_mask.sum(dim=-1)

tensor([509, 512, 511], device='cuda:0')

In [23]:
inp_id = generated_outputs[:1, :]
attention_mask_one = attention_mask[:1, :]

In [24]:
ans = model(input_ids = inp_id, attention_mask = attention_mask_one)

In [7]:
import torch.nn as nn


class Agent(nn.Module):
  def __init__(self,model,embd_dim, dropout=0.0):
    super().__init__()
    self.model = model
    self.value_head = nn.Sequential(
            nn.LayerNorm(embd_dim),
            nn.GELU(),
            nn.Linear(embd_dim, 4*embd_dim),
            nn.GELU(),
            nn.Linear(4*embd_dim, 1),
        )

  def generate(self, **x):
    return self.model.generate(**x, max_new_tokens=512)


  def forward(self,x):
    out = self.model(x, output_hidden_states=True).hidden_states # this outputs hidden state for each transformer layer
    last_hidden_state = out[-1] #(B,T,C)
    logits = self.model.lm_head(last_hidden_state) # (B,T,vocab_length)

    value = self.value_head(last_hidden_state) #(B,T,1)
    return logits, value


In [42]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [19]:
model.config

OPTConfig {
  "_attn_implementation_autoset": true,
  "_remove_final_layer_norm": false,
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": true,
  "dropout": 0.1,
  "enable_bias": true,
  "eos_token_id": 2,
  "ffn_dim": 8192,
  "hidden_size": 2048,
  "init_std": 0.02,
  "layer_norm_elementwise_affine": true,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 32,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 50272,
  "word_embed_proj_dim": 2048
}

In [8]:
agent = Agent(model,model.config.hidden_size).to(device)

In [160]:
def tokenize_fn(item):
  tokenized_data = tokenizer(item['text'], return_tensors='pt', add_special_tokens=True, max_length=40, truncation=True)
  item['input_ids'] = tokenized_data['input_ids'].squeeze()
  item['attention_mask'] = tokenized_data['attention_mask'].squeeze()

  return item


In [109]:
imdb_dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [161]:
exp_db = Dataset.from_dict(imdb_dataset['train'][:20])
exp_tokenized = exp_db.map(tokenize_fn)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [167]:
exp_tokenized['attention_mask'][:2]

sample_input = {'input_ids' : torch.tensor(exp_tokenized['input_ids'][:2]), 'attention_mask' : torch.tensor(exp_tokenized['attention_mask'][:2])}

In [172]:
sample_input['input_ids'].shape

torch.Size([2, 40])

In [93]:
from dataclasses import dataclass

@dataclass
class Config:
  rollout_iterations: int = 2000
  model_name: str = "facebook/opt-350m"
  epochs: int = 1
  num_steps: int = 100
  batch_size : int = 2
  max_new_tokens : int = 600
  temperature: float = 0.7
  kl_coeff : float = 0.1
  gamma : float = 0.95
  lambda_v : float = 0.99
  clip_epsilon: float = 0.2, # the surrogate clipping coefficient
  ent_coef: float = 0.01, # entroy coefficient controlling the exploration factor C2
  vf_coef: float = 0.5, # value function controlling value estimation importance C1

config = Config()

In [81]:
import torch
def get_data(data):
  indexes = torch.randint(len(data), (config.batch_size,))
  return data[indexes]

In [84]:
# make something to save the generated tokens.
from datasets import Dataset


class Trajectories(Dataset):
  def __init__(self, logprobs=None, actions=None, rewards=None, values=None, advantages=None, end=None, returns=None):
    self.actions = actions
    self.rewards = rewards
    self.values = values
    self.advantages = advantages
    self.logprobs = logprobs
    self.end = end # end of generation, after this index all tokens are pad token
    self.returns = returns
  def __getitem__(self,idx):
    return self.logprobs[idx], self.actions[idx], self.rewards[idx], self.values[idx], self.advantages[idx],self.end[idx], self.returns

  def __len__(self):
    return len(self.logprobs)

  def add(self,logprobs, actions, rewards, values, advantages, end):
    self.actions = torch.cat((self.actions, actions), dim=0) if self.actions is not None else actions
    self.rewards = torch.cat((self.rewards, rewards), dim=0) if self.rewards is not None else rewards
    self.values = torch.cat((self.values, values), dim=0) if self.values is not None else values
    self.advantages = torch.cat((self.advantages, advantages), dim=0) if self.advantages is not None else advantages
    self.logprobs = torch.cat((self.logprobs, logprobs), dim=0) if self.logprobs is not None else logprobs
    self.end = torch.cat((self.end, end), dim=0) if self.end is not None else end
    self.returns = torch.cat((self.returns, returns), dim=0) if self.returns is not None else returns



In [104]:
trajectories = agent.generate(**inputs)

In [13]:
with torch.no_grad():
  logits, values = agent(trajectories.to(device))
  ref_model_logits = model(trajectories)

In [14]:
logits.shape

torch.Size([3, 517, 50272])

In [16]:
B,T = trajectories.shape
B,T

(3, 517)

In [17]:
rewards = torch.randint(100, (B,), dtype=torch.long).to(device)

In [18]:
rewards

tensor([77, 34, 82], device='cuda:0')

In [19]:
logits = torch.log_softmax(logits, dim=-1)
logprobs = torch.gather(logits[:,:-1,:], dim =-1, index = trajectories[:,1:].unsqueeze(dim=-1)).squeeze()

In [20]:
ref_model_logits = torch.log_softmax(ref_model_logits.logits, dim=-1)
ref_model_logprobs = torch.gather(ref_model_logits[:,:-1,:], dim =-1, index = trajectories[:,1:].unsqueeze(dim=-1)).squeeze()

In [21]:
attention_mask = trajectories.not_equal(tokenizer.pad_token_id).long()

In [22]:
start = inputs['input_ids'].shape[1] - 1
start

4

In [23]:
end = attention_mask[:,start+1:].sum(dim=-1)
end

tensor([512, 512, 512], device='cuda:0')

In [24]:

token_rewards = -0.1 * logprobs - ref_model_logprobs

In [25]:
end

tensor([512, 512, 512], device='cuda:0')

In [26]:
token_rewards

tensor([[7.6187e+00, 2.9436e+00, 2.5706e-03,  ..., 6.0164e-03, 1.1513e-04,
         1.4091e-03],
        [4.7907e+00, 2.8869e+00, 8.0073e+00,  ..., 1.3325e-03, 8.0655e-04,
         2.2269e-01],
        [8.9809e+00, 2.0089e+00, 1.3762e+00,  ..., 1.0015e-02, 2.2749e-04,
         6.1124e-03]], device='cuda:0')

In [29]:
for t in range(B):
  # token_rewards[t, start + end[t]] = torch.tensor(rewards[t]).clone().detach().float()
  token_rewards[t, start + end[t] -1] = rewards[t].float()

In [32]:
token_rewards.shape

torch.Size([3, 516])

In [35]:
values = values[:,:-1].squeeze()

In [39]:
values.shape

torch.Size([3, 516])

In [40]:
config

Config(rollout_iterations=2000, model_name='facebook/opt-350m', epochs=1, num_steps=100, batch_size=8, max_new_tokens=600, temperature=0.7, kl_coeff=0.1)

In [46]:
def gae(rewards, values):
  last_value = 0
  last_advantage = 0
  advantages = torch.zeros_like(values).to(rewards.device)

  for t in reversed(range(rewards.shape[1])):
    delta = rewards[:,t] + config.gamma*last_value - values[:,t]
    advantages[:,t] = delta + config.lambda_v* config.gamma*last_advantage
    last_value = values[:,t]
    last_advantage = advantages[:,t]

  returns = advantages + values

  return advantages, returns

In [49]:
advantages, returns = gae(token_rewards, values)

In [50]:
advantages.shape

torch.Size([3, 516])

In [60]:
trajectories.shape

torch.Size([3, 516])

In [78]:
end

tensor([512, 512, 512], device='cuda:0')

In [105]:
rollouts = Trajectories()

rollouts.add(trajectories, token_rewards, values, advantages, logprobs, end,returns)

In [106]:
retrieved_data = get_data(rollouts)

In [107]:
(retrieved_data[-1])

tensor([512, 512], device='cuda:0')

In [108]:
## NOW lets do the generation and the update.

# we generate from the updated model.

r_actions, r_rewards, r_values, r_advantages, r_logprobs, r_end, r_returns = retrieved_data

In [109]:
# since we want to train this model, we don't do torch.no_grad() here,

new_model_logits, new_model_values = agent(r_actions)
new_mode_logits = torch.log_softmax(new_model_logits, dim =-1)



# now we have everything we needd



In [100]:
new_mode_logits.shape

torch.Size([2, 516, 50272])

In [110]:
new_model_logprobs = torch.gather(new_model_logits[:,:-1, :], dim=-1, index=r_actions[:,1:].unsqueeze(-1)).squeeze() # will be 516

In [111]:
new_model_logprobs.shape

torch.Size([2, 516])

In [112]:
loss_mask = torch.zeros_like(new_model_logprobs)

for t in range(config.batch_size):
  # generate loss mask from start and end

  loss_mask[t: start: end[t]] = 1


new_model_logprobs_flatten = (new_model_logprobs * loss_mask).view(-1)
new_model_values_flatten = (new_model_values * loss_mask).view(-1)
r_logprobs_flatten = (r_logprobs * loss_mask).view(-1)
r_advantages_flatten = (r_advantages * loss_mask).view(-1)

ppo_loss = loss_clip(r_logprobs_flatten, new_model_logprobs_flatten, r_advantages_flatten)
value_func_loss = loss_vf( r_returns, new_model_values_flatten)

loss = ppo_loss + value_func_loss
l
# define the optimizer
# loss backward
# optimizer step

# that should be it, recheck everything once,



torch.Size([2, 516])

In [ ]:
def loss_vf(
    mb_oldreturns,  # mini batch of old returns collected during the rollout
    mb_newvalues    # minibach of values calculated by the new value function
):
    """
    enforcing the value function to give more accurate estimates of returns
    """
    mb_newvalues = mb_newvalues.view(-1)
    loss = 0.5 * ((mb_newvalues - mb_oldreturns) ** 2).mean()
    return loss

In [92]:
def loss_clip(
    mb_oldlogporb,     # old logprob of mini batch actions collected during the rollout
    mb_newlogprob,     # new logprob of mini batch actions created by the new policy
    mb_advantages      # mini batch of advantages collected during the the rollout
):
    """
    policy loss with clipping to control gradients
    """
    ratio = torch.exp(mb_newlogprob - mb_oldlogporb)
    policy_loss = -mb_advantages * ratio
    # clipped policy gradient loss enforces closeness
    clipped_loss = -mb_advantages * torch.clamp(ratio, 1 - configs.clip_epsilon, 1 + configs.clip_epsilon)
    pessimistic_loss = torch.max(policy_loss, clipped_loss).mean()
    return pessimistic_loss

In [ ]:
def rollout(tokenized_dataset):
  # Generate samples from the tokenized_dataset, and calculate v,r,a
  for i in range(config.rollout_iterations):
    to_generate_data = get_data(tokenized_dataset)

    trajectories = agent.generate(**to_generate_data)
    logits, values = agent(trajectories) # (B,T,vocab_len), (B,T,1))
    rewards = torch.zeros(values.shape[0],values.shape[1])

    # now find the reward,
    output = tokenizer.decode(generated_data)
    # mask = to_generate_data
    total_reward = reward_model(output) # B


    #incorporate kl penalty
    # reference model
    ref_model_data = model.generate(**to_generate_data, max_new_tokens=config.max_new_tokens, output_logits=True, return_dict_in_generate=True) #(B,T)
    ref_model_probs = torch.log_softmax(ref_model_data.logits, dim=-1)
    ref_data_logprobs = torch.gather(ref_model_probs, dim=-1, index=ref_model_data) # get the logprobs for generated data

    # training model
    probs = torch.log_softmax(logits, dim=-1)
    new_data_logprobs = torch.gather(probs, dim=-1, index=ref_model_data)

    rewards =  new_data_logprobs - ref_data_logprobs

    # advantage GAE







